In [1]:
import pandas as pd
import numpy as np

In [8]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
df_from_csv = pd.read_csv('dataset/auction_data_added_testing.csv')
# df_from_csv = pd.read_csv('dataset/auction_data_added_training.csv')
df_from_csv.fillna(0, inplace=True) # fill for auctions with only 1 bid

## Explore Data

In [9]:
df_from_csv.head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid
0,0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.157894736,0.0,6.594607,10.631530,0.000000,0.631579,2.473684,8.052632,111.105263,2.473684,19.243910,2068,1,0
1,1,14,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,wberf,jewelry,phone15,9759243263157894,au,134.243.148.253,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.263157894,0.0,4263.473684,4448.048902,1249.210526,1709.157895,2169.105263,5770.605263,9372.105263,2169.105263,0.000000,4,1,0
2,2,106,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,hw5at,jewelry,phone80,9759243736842105,ke,227.231.81.211,v2gyrh0dbr31atn,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.736842105,0.0,48.986369,145.055987,0.052632,7.223684,19.105263,41.565789,2080.315789,19.105263,143.252999,279,1,0
3,3,152,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,uw3sx,jewelry,phone41,9759243947368421,iq,140.231.164.76,xdsfv4oevw5owkq,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.947368421,0.0,9.655037,12.437212,0.000000,2.052632,5.526316,12.736842,121.842105,5.526316,18.918137,1410,1,0
4,4,155,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,q79ya,jewelry,phone102,9759244000000000,za,187.81.242.103,eex5it5avrad6s8,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:04.000000000,0.0,15.387376,35.667397,0.000000,2.052632,6.473684,16.236842,477.368421,6.473684,98.942022,548,1,0


In [10]:
df_from_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4585110 entries, 0 to 4585109
Data columns (total 26 columns):
index                                  int64
bid_id                                 int64
bidder_id                              object
auction                                object
merchandise                            object
device                                 object
time                                   int64
country                                object
ip                                     object
url                                    object
payment_account                        object
address                                object
date_time                              object
time_delta_auction_bids                float64
time_delta_auction_bids_mean           float64
time_delta_auction_bids_std            float64
time_delta_auction_bids_min            float64
time_delta_auction_bids_25%            float64
time_delta_auction_bids_50%            float64
time_delta

In [11]:
# bots = df_from_csv.loc[df_from_csv['outcome'] == 1]
bots = pd.read_csv('dataset/bots.csv')
bots.fillna(0, inplace=True) # fill for auctions with only 1 bid
bot_count = len(bots['bidder_id'].unique())
bot_ratio = bot_count / len(df_from_csv['bidder_id'].unique())
bot_bids = bots.shape[0]
bot_bids_ratio = bot_bids / df_from_csv.shape[0]
# bots.to_csv('dataset/bots.csv', index=None)

In [12]:
print('bots: {}, % {:.2f} of total bidders are bots for training'.format(bot_count, bot_ratio))
print('bot bids: {}, % {:.2f} of total bot bids for training'.format(bot_bids, bot_bids_ratio))

bots: 103, % 0.02 of total bidders are bots for training
bot bids: 412416, % 0.09 of total bot bids for training


In [13]:
for column in bots.columns:
    print(column + ': {}'.format(len(bots[column].unique())))

index: 412416
bid_id: 412416
bidder_id: 103
auction: 6390
merchandise: 7
device: 3078
time: 313212
country: 192
ip: 230125
url: 54865
payment_account: 103
address: 103
outcome: 1
date_time: 313212
time_delta_auction_bids: 16493
time_delta_auction_bids_mean: 6171
time_delta_auction_bids_std: 5991
time_delta_auction_bids_min: 1463
time_delta_auction_bids_25%: 4627
time_delta_auction_bids_50%: 4766
time_delta_auction_bids_75%: 5678
time_delta_auction_bids_max: 6079
time_delta_bids_in_auction_median: 4766
time_delta_bids_in_auction_kurtosis: 5714
total_bids_in_auction: 1175
first_auction_bid: 2
last_auction_bid: 2


## get single bidder to test with

In [16]:
df = df_from_csv.loc[df_from_csv['bidder_id'] == '8dac2b259fd1c6d1120e519fb1ac14fbqvax8'].copy(deep=True)

# add temporal difference between bids
df['date_time'] = pd.to_datetime(df['time'])

df = df.sort_values(['date_time']) # must sort by date some funny stuff with index happens

tdelta = df['date_time'].diff()
tdelta.fillna(0 , inplace=True)
df['time_delta_user_bids'] = pd.to_timedelta(tdelta) / np.timedelta64(1, 's')

df.shape

(62529, 27)

In [17]:
df.head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid,time_delta_user_bids
0,0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.157894736,0.0,6.594607,10.631530,0.000000,0.631579,2.473684,8.052632,111.105263,2.473684,19.243910,2068,1,0,0.000000
1,1,14,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,wberf,jewelry,phone15,9759243263157894,au,134.243.148.253,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.263157894,0.0,4263.473684,4448.048902,1249.210526,1709.157895,2169.105263,5770.605263,9372.105263,2169.105263,0.000000,4,1,0,0.105263
2,2,106,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,hw5at,jewelry,phone80,9759243736842105,ke,227.231.81.211,v2gyrh0dbr31atn,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.736842105,0.0,48.986369,145.055987,0.052632,7.223684,19.105263,41.565789,2080.315789,19.105263,143.252999,279,1,0,0.473684
3,3,152,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,uw3sx,jewelry,phone41,9759243947368421,iq,140.231.164.76,xdsfv4oevw5owkq,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:03.947368421,0.0,9.655037,12.437212,0.000000,2.052632,5.526316,12.736842,121.842105,5.526316,18.918137,1410,1,0,0.210526
4,4,155,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,q79ya,jewelry,phone102,9759244000000000,za,187.81.242.103,eex5it5avrad6s8,a3d2de7675556553a5f08e4c88d2c2282ay1v,7f4c6ac489cbd95e38e4f093bb308af5jae25,1970-04-23 22:54:04.000000000,0.0,15.387376,35.667397,0.000000,2.052632,6.473684,16.236842,477.368421,6.473684,98.942022,548,1,0,0.052632


In [18]:
df[1:]['time_delta_user_bids'].median(), df[1:]['time_delta_user_bids'].mean(), df[1:]['time_delta_user_bids'].kurtosis()

(0.105263158, 0.21816415719438717, 24.411030007021491)

In [19]:
# get describe series (not including first row delta will always be 0)
describe_of_time_between_bids = df[1:].describe()['time_delta_user_bids']
describe_of_time_between_bids

count    62528.000000
mean         0.218164
std          0.329413
min          0.000000
25%          0.052632
50%          0.105263
75%          0.263158
max          5.947368
Name: time_delta_user_bids, dtype: float64

In [20]:
# get axes labels for label describe
describe_axes = describe_of_time_between_bids.axes[0]
describe_axes

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [21]:
# add distrubtion of deltas data
for i in range(1, len(describe_axes)):
    df['{}_{}'.format('time_delta_user_bids', describe_axes[i])] = describe_of_time_between_bids[i]

In [22]:
bot_ip = set(bots['ip'].unique())

In [23]:
len(df['ip'].unique()), len(bot_ip)

(21741, 230125)

In [24]:
count = 0
for ip in df['ip']:
    if ip in bot_ip:
        count += 1
count / df.shape[0]

0.6114123046906236

## slice df by bidder_id

In [25]:
bidder_dictinoary = {}
bidders = df_from_csv['bidder_id'].unique()
count = 1

for bidder in bidders:
    bidder_dictinoary[bidder] = df_from_csv.loc[df_from_csv['bidder_id'] == bidder]
    
    print('count: {}, id: {}, df_shape: {}'.format(count, bidder, bidder_dictinoary[bidder].shape))
    count += 1

count: 1, id: 8dac2b259fd1c6d1120e519fb1ac14fbqvax8, df_shape: (62529, 26)
count: 2, id: aa5f360084278b35d746fa6af3a7a1a5ra3xe, df_shape: (105436, 26)
count: 3, id: 3939ac3ef7d472a59a9c5f893dd3e39fh9ofi, df_shape: (10956, 26)
count: 4, id: 8393c48eaf4b8fa96886edc7cf27b372dsibi, df_shape: (12888, 26)
count: 5, id: eef4c687daf977f64fc1d08675c44444raj3s, df_shape: (893, 26)
count: 6, id: ab056855c9ca9d36390feae1fa485883issyg, df_shape: (6, 26)
count: 7, id: d600dc03b11e7d782e1e4dae091b084a1h5ch, df_shape: (3599, 26)
count: 8, id: fe37b8b29d26021ea9462bcc7c5bf5c2pbya7, df_shape: (6232, 26)
count: 9, id: 990db0dcdb665fb8fda5845b3e5d63082mbqq, df_shape: (30414, 26)
count: 10, id: 1a9c554b044c77239b63636008d353d4vy9kh, df_shape: (2316, 26)
count: 11, id: ffbc0fdfbf19a8a9116b68714138f2902cc13, df_shape: (25075, 26)
count: 12, id: c4856fd5abe8f6d6dea36ca2fec444faauos8, df_shape: (14695, 26)
count: 13, id: d174177146370aacbd5ae107b6be78828tx3e, df_shape: (23651, 26)
count: 14, id: 6b09f3376045eb

count: 111, id: 6d10b04b5480a655fed72b9a812f5b4bpqvhd, df_shape: (1388, 26)
count: 112, id: 5e9fae8fb44a50c285436949f7f00460wjvsb, df_shape: (4808, 26)
count: 113, id: 4126bfd1c612699bd521b9499bb4796d74y9x, df_shape: (4414, 26)
count: 114, id: 78e211d99d5bc69c1ffc537948ebb864a8wq7, df_shape: (1301, 26)
count: 115, id: 304d579f950e6a0c4f8f4bd9e712cb7emv1is, df_shape: (501, 26)
count: 116, id: f4ad276544656ed2182af92e3b88dfba8l3l9, df_shape: (2003, 26)
count: 117, id: c3d567f4af0d49fa9ddbdeea3bb0445f0ablr, df_shape: (2258, 26)
count: 118, id: c0bf5fd69c39b37e64a2feb0f45788419hqx5, df_shape: (1683, 26)
count: 119, id: d30fd9b4ea5444281539670f3da5cd880j572, df_shape: (2683, 26)
count: 120, id: fb3620e1a7ecd3d00ad46538ab2f4598whezy, df_shape: (1874, 26)
count: 121, id: 867ae2212dcf4815255cfc5772d1a4e1vcfpa, df_shape: (1293, 26)
count: 122, id: 1ba5b20f16d914e3d76652d5c9cdcbf5iazkd, df_shape: (2086, 26)
count: 123, id: c5d3417082e10d448bbdb1e9ebb6d6d2sng1z, df_shape: (450, 26)
count: 124, id

count: 219, id: a50f58ceb34c347c33e8b22f4470d928mwjew, df_shape: (2097, 26)
count: 220, id: 7900c9dc6bc29cacd7ec41e18467e1f2el6lv, df_shape: (6357, 26)
count: 221, id: 59511585dfd45b73e7899d1f1908eaa2d26dl, df_shape: (1145, 26)
count: 222, id: f071693216659f048d90155a3bcf37a1pcvzf, df_shape: (2543, 26)
count: 223, id: 7f6d1b6369786181188848e7903ba72ckc0xv, df_shape: (746, 26)
count: 224, id: 96d19fea6299dfabe564c5a58bc716cbl3rvh, df_shape: (2, 26)
count: 225, id: 9dddb2f6c99dad940da2227d700c9e5btcmkw, df_shape: (2223, 26)
count: 226, id: 17426ea6a78b89ab83bf1d5c2df04833wwrff, df_shape: (355, 26)
count: 227, id: ce82fd4ac6e3ba449bad1e29f039203agke56, df_shape: (697, 26)
count: 228, id: 6f01c129a72deecf0b14c28b045c6f45bugqo, df_shape: (319, 26)
count: 229, id: c0abe67b20c4e4165d8e4d319d6db6d0w5b87, df_shape: (29, 26)
count: 230, id: 6091a98b1b777858f76ed61da3b0bbe0jqwz6, df_shape: (135, 26)
count: 231, id: 435f3414afda395f6220fc2da77d0cde4ma0p, df_shape: (1456, 26)
count: 232, id: 5e33ae

count: 329, id: 02518a364d87661eed1453e251933862yvmjj, df_shape: (749, 26)
count: 330, id: 0764308c5a1d35bf76c32d943eff07a50tp9e, df_shape: (45, 26)
count: 331, id: ab7cc752f0ed98d7f8c85e02dbbc4772z9xpx, df_shape: (56, 26)
count: 332, id: 7e4a99f7ae5fe3a098c9c3f21b9a5cbfa4w4t, df_shape: (437, 26)
count: 333, id: 3de7d3030fecf543874081f7ab0a248evqper, df_shape: (410, 26)
count: 334, id: b8fc67ed5825409a40adca46ee7f053auho77, df_shape: (19, 26)
count: 335, id: 6951db6168b80060a103ab65d0e074484c1vc, df_shape: (568, 26)
count: 336, id: 9a51d309c8824bae38bd75eddce1a99fe208r, df_shape: (260, 26)
count: 337, id: b02df28d2798f0b6c2b1112a9d407ae7wbj8r, df_shape: (777, 26)
count: 338, id: a9c0cd61e6b52a5b4bfb88e5ccb40af36qzu2, df_shape: (8, 26)
count: 339, id: 576c9f2539f29b0775794d71802cc7f6qhcg6, df_shape: (361, 26)
count: 340, id: 91d917d9f7d2d83da3b7347d942b8e28tc8tv, df_shape: (819, 26)
count: 341, id: 4a8033e0cd9575560c86920b99d67f427hgge, df_shape: (264, 26)
count: 342, id: c84d73420fa4dd

count: 439, id: 15ad4021a5480aa57cf9e69d75907e47wprdz, df_shape: (261, 26)
count: 440, id: f6fa31944ec345bd95de03e739ff0f43mgxjz, df_shape: (94, 26)
count: 441, id: a4159bd8aa351a5e46250063bf9d22e2atrr4, df_shape: (5, 26)
count: 442, id: 5a28b19b882d307946b8d8f67198c9b76xpqn, df_shape: (144, 26)
count: 443, id: 6ef33633feca956e287a081cec6953ad0djvi, df_shape: (376, 26)
count: 444, id: 77c529bf6ca232649b7df602279b0e0c6xcax, df_shape: (485, 26)
count: 445, id: 0051aef3fdeacdadba664b9b3b07e04e4coc6, df_shape: (68, 26)
count: 446, id: b687d9dedf1e202034b739ddbb87bbf8rovcr, df_shape: (135, 26)
count: 447, id: 945c7f4a9d8780193f262bd8d61378c49fbrw, df_shape: (3647, 26)
count: 448, id: 7b7d703a6ff871d4a379bf182a6932c0xoqga, df_shape: (8, 26)
count: 449, id: f35e1c29147c2618ed2d4e7ed88b0c8eivimx, df_shape: (731, 26)
count: 450, id: a96978151b02d9254edcd77d6276b0889frnl, df_shape: (146, 26)
count: 451, id: b34feb72d3ad58fd605a340859c7639csfnfh, df_shape: (30, 26)
count: 452, id: 2fb5873c7573949

...

count: 4173, id: 4abedb0c05a6b24ab5561fb83906c81endh4u, df_shape: (1, 26)
count: 4174, id: 66bb4d936cd2df2fb91200c082bc9051gigr8, df_shape: (89166, 26)
count: 4175, id: 5f7b3e3d8c7bef8e25d9bed005993e6ewj4gs, df_shape: (1423, 26)
count: 4176, id: 2dc9f39de8c5e3e877dfbb7223bb24damfdcs, df_shape: (1777, 26)
count: 4177, id: d1d8bb8112afd9cc05dd89218e91a914474m4, df_shape: (3792, 26)
count: 4178, id: 2f8792bfac20d2403f7da914791310fcmr7n8, df_shape: (293, 26)
count: 4179, id: efc4e5829f0ce82b6bfd059c61e12380gs08h, df_shape: (9, 26)
count: 4180, id: ee80643a10e36febd94f0b5e841c3b68szqxo, df_shape: (2501, 26)
count: 4181, id: a30535768c8f884cf1d84cf8b778232d2wp3x, df_shape: (837, 26)
count: 4182, id: 43303f008c8bbad1f83695155a4cdcaenrt6d, df_shape: (2444, 26)
count: 4183, id: b0b7a92d73f679521097bd885fde3c80k2n0z, df_shape: (2562, 26)
count: 4184, id: a4ebed147d656108e306167b2fce5bd32oi58, df_shape: (808, 26)
count: 4185, id: e96861eb35f6571da0c77d2bca822782whvn6, df_shape: (186, 26)
count: 4

count: 4282, id: 0550568ad77e8e41f2b188be324c10b8xw912, df_shape: (2, 26)
count: 4283, id: 99cddad0443cc404fb95bf2758f1e75c64am4, df_shape: (1, 26)
count: 4284, id: eb2ffc4eb9d2c03a9aa1414bb69f021erzz7e, df_shape: (21, 26)
count: 4285, id: 4f36ffebcfe248658df4388e7826936ejjja1, df_shape: (44, 26)
count: 4286, id: 6c9cecb80574533f1deabc943d7c5ed0oxi5q, df_shape: (1, 26)
count: 4287, id: 81f09434ecc54f09ec8399678a5b3c3a67o1e, df_shape: (9, 26)
count: 4288, id: 9167c4d2005c6e55bd7b3c1fe79e3fbd53hjl, df_shape: (2, 26)
count: 4289, id: 4ccde027a20264ab2d04392b60e31b4fu3ray, df_shape: (2, 26)
count: 4290, id: a356163fe7434d47b1c6480c87366394oaibe, df_shape: (2, 26)
count: 4291, id: da8abe1879eb50004c6276c94b792e11lq4xn, df_shape: (7, 26)
count: 4292, id: a852b00d87721f8d3ca2e1041c088aa507iso, df_shape: (88, 26)
count: 4293, id: 58cbeb46c36b90a723cded01495d055bmwudo, df_shape: (2, 26)
count: 4294, id: 10a1f1cb1fdd93a139efba25087dbeeaulogd, df_shape: (2, 26)
count: 4295, id: 980a1950c0b5df2b9e

count: 4393, id: cc502c2ea229d364503dd18cda71d30bklt3h, df_shape: (3, 26)
count: 4394, id: 0c7f27623e1ed809d8203bfa54e7b0f1438cg, df_shape: (12, 26)
count: 4395, id: 470af95570ebd931c4d9dd564e005de9a7sdv, df_shape: (1, 26)
count: 4396, id: c726caee0e311e7b05d459debbcd1e82woxaq, df_shape: (1, 26)
count: 4397, id: 25dba4cb8c9e45c18dfdf7140be99d7f0ffb0, df_shape: (3, 26)
count: 4398, id: 47a16c48b321926385db94b5a302642awmq11, df_shape: (63, 26)
count: 4399, id: 25bc7e9c166259c232c1bece2fde87e8bhtnk, df_shape: (1, 26)
count: 4400, id: 4b9260b578bac8d676cb8d1d7b90c553zgisk, df_shape: (5, 26)
count: 4401, id: 9e94358616ff4aa21734608ae761ccf5cfp6d, df_shape: (118, 26)
count: 4402, id: 337f4ff4946d4de1cf4ae216c1ff5c88e7oms, df_shape: (2, 26)
count: 4403, id: 92f2c8d4d96d95a60a0e7cc67f9e815bhqjpn, df_shape: (3, 26)
count: 4404, id: 08144977af383cb9f1d3b51ade19f35frjmwm, df_shape: (1, 26)
count: 4405, id: 1cf7c4ce75a054e0b036f8d5d33392394458z, df_shape: (1, 26)
count: 4406, id: 9d294227f45e27732

count: 4504, id: 50a7a76378deb17240950c1915f45e11yf97z, df_shape: (1, 26)
count: 4505, id: cb3314eb25be845fa412f1d2c2934841jmaqj, df_shape: (3, 26)
count: 4506, id: 0695b948daa6f8bffb8758d201c27517k50ao, df_shape: (1, 26)
count: 4507, id: 24ec8762d6a57f212ba2cfb2e59cdf5ejd01s, df_shape: (1, 26)
count: 4508, id: 139f2fbd02e92211aeb48b5882525db2t3hyw, df_shape: (1, 26)
count: 4509, id: 3be692e6a79ba53f596e55cb68592db7rj901, df_shape: (16, 26)
count: 4510, id: ba3070025ca7622e20e71feddd2a94710vlpx, df_shape: (3, 26)
count: 4511, id: 920796b1e91e5740c589ea274cd9dca3c2c13, df_shape: (291, 26)
count: 4512, id: 96145a3b21bf25e866844540be17251dn0v0w, df_shape: (2, 26)
count: 4513, id: 159fe5a279ff287a212320cedde77eb7kr0a4, df_shape: (1, 26)
count: 4514, id: b22726dcf17e400c21bf59505d25bb4491b5e, df_shape: (2, 26)
count: 4515, id: 06bbde5ca852af9ed6f2afdc6ee5bc3bko1h1, df_shape: (1, 26)
count: 4516, id: 353355147a9e15ea7b8b6588278b609cz1r26, df_shape: (2, 26)
count: 4517, id: ca50daf4dc80460b5a

count: 4614, id: 61665d0c0ac353aa4f429f636879af8bmuf9n, df_shape: (1, 26)
count: 4615, id: 3a2e6d8da93da7b6f5b9c6ba770f8687tfkyk, df_shape: (1, 26)
count: 4616, id: 1af3a143e126acfc43031dd8e4612b2egr927, df_shape: (1, 26)
count: 4617, id: 83b6cbf8c26ea604b48f508dd7728c9414pfn, df_shape: (1, 26)
count: 4618, id: bd890937b5601111119969b45bde5056eycor, df_shape: (1, 26)
count: 4619, id: 107f0f8f2bf4fc62ba8c01003c516c03npt0l, df_shape: (1, 26)
count: 4620, id: 9f9313a1d8a88b3a3c8a2d893a92820dn38qc, df_shape: (1, 26)
count: 4621, id: 5563598101c7e6533b63f26827e57aa96qjww, df_shape: (1, 26)
count: 4622, id: 15e7068e841df3f00857b9b1db46e79200trv, df_shape: (3, 26)
count: 4623, id: 5a8019da4075c77405e43efd072670eegbme0, df_shape: (1, 26)
count: 4624, id: 9390ac0a97e75b3b56fe63695d27a82euhv69, df_shape: (1, 26)
count: 4625, id: 13eaa629e1a8fd178403a66fcf0758bbirawk, df_shape: (1, 26)
count: 4626, id: d96919260372faba6c766a734e5af2f0bmnrx, df_shape: (1, 26)
count: 4627, id: ab322943595e795187896

In [29]:
len(bidder_dictinoary)

4630

In [30]:
bidder_dictinoary['eef4c687daf977f64fc1d08675c44444raj3s'].head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid
191809,191809,6,eef4c687daf977f64fc1d08675c44444raj3s,kjlzx,mobile,phone2,9759243210526315,th,152.235.155.159,j9nl1xmo6fqhcc6,a3d2de7675556553a5f08e4c88d2c228f4wpj,a3d2de7675556553a5f08e4c88d2c2288h7pg,1970-04-23 22:54:03.210526315,0.000000,51.522672,71.139020,0.052632,12.776316,32.342105,56.723684,526.473684,32.342105,18.560097,261,1,0
191810,191810,4781,eef4c687daf977f64fc1d08675c44444raj3s,yuokv,mobile,phone17,9759267210526315,th,129.175.199.35,d132n7xgqxkjdiv,a3d2de7675556553a5f08e4c88d2c228f4wpj,a3d2de7675556553a5f08e4c88d2c2288h7pg,1970-04-23 22:54:27.210526315,0.000000,11.192001,16.216468,0.000000,2.368421,6.105263,14.052632,168.368421,6.105263,23.646949,1217,1,0
191811,191811,9472,eef4c687daf977f64fc1d08675c44444raj3s,i1aya,mobile,phone63,9759278842105263,ch,104.166.176.232,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228f4wpj,a3d2de7675556553a5f08e4c88d2c2288h7pg,1970-04-23 22:54:38.842105263,0.052632,0.625295,42.462515,0.000000,0.052632,0.105263,0.210526,3788.736842,0.105263,7875.171291,15755,0,0
191812,191812,11453,eef4c687daf977f64fc1d08675c44444raj3s,ba53g,mobile,phone185,9759284368421052,th,99.55.255.151,3uo2cm4ryjri5rz,a3d2de7675556553a5f08e4c88d2c228f4wpj,a3d2de7675556553a5f08e4c88d2c2288h7pg,1970-04-23 22:54:44.368421052,0.000000,93.997301,177.502983,1.631579,13.315789,34.315789,75.368421,931.368421,34.315789,14.252567,40,1,0
191813,191813,11668,eef4c687daf977f64fc1d08675c44444raj3s,im3yk,mobile,phone82,9759284894736842,mg,49.200.95.239,i20cb2wksfwjv4s,a3d2de7675556553a5f08e4c88d2c228f4wpj,a3d2de7675556553a5f08e4c88d2c2288h7pg,1970-04-23 22:54:44.894736842,0.105263,0.754770,28.578513,0.000000,0.105263,0.263158,0.578947,2843.631579,0.263158,8293.579443,15229,0,0


## add features generated from testing on single bidder to all bidder

In [31]:
# path = 'user_bids_dfs_training'
path = 'user_bids_dfs_testing'
i = 0

bot_ip = set(bots['ip'].unique())


for bidder in bidders:
    
    df = bidder_dictinoary[bidder]
    
    df['date_time'] = pd.to_datetime(df['time'])

    df = df.sort_values(['date_time']) # must sort by date some funny stuff with index happens

    # add temporal difference between bids
    tdelta = df['date_time'].diff()
    tdelta.fillna(0 , inplace=True)
    df['time_delta_user_bids'] = pd.to_timedelta(tdelta) / np.timedelta64(1, 's')

    # get describe series (not including first row delta will always be 0)
    describe_of_time_between_bids = df[1:]['time_delta_user_bids'].describe()
    describe_of_time_between_bids

    # get axes labels for label describe
    describe_axes = describe_of_time_between_bids.axes[0]
    describe_axes

    # add distrubtion of deltas data
    for i in range(1, len(describe_axes)):
        df['{}_{}'.format('time_delta_user_bids', describe_axes[i])] = describe_of_time_between_bids[i]
        
    df['time_delta_user_bids_median'] = df[1:]['time_delta_user_bids'].median()
    df['time_delta_user_bids_kurtosis'] = df[1:]['time_delta_user_bids'].kurtosis()
                                              
    count = 0
    for ip in df['ip']:
        if ip in bot_ip:
            count += 1
            
    df['bids_with_bot_ip']  = count # bids that shared ip with bot
    
    df['total_bids'] = df.shape[0] # total bid count
                                              
    user_ip = set(df['ip'].unique())
                                              
    df['ip_shared_with_bots'] = len(bot_ip & user_ip) # number of unqiue ips that are bot ip
                                              
    df['unique_ip'] = len(user_ip) # unqiue user ip
                                              
    df['avg_bids_per_url'] = df.shape[0] / len(df['url'].unique())
    
    df.to_csv('{}/{}.csv'.format(path,bidder), index=None)
     
                                  
    print('biders saved: {}'.format(bidder))

/home/luis/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


biders saved: 8dac2b259fd1c6d1120e519fb1ac14fbqvax8
biders saved: aa5f360084278b35d746fa6af3a7a1a5ra3xe
biders saved: 3939ac3ef7d472a59a9c5f893dd3e39fh9ofi
biders saved: 8393c48eaf4b8fa96886edc7cf27b372dsibi
biders saved: eef4c687daf977f64fc1d08675c44444raj3s
biders saved: ab056855c9ca9d36390feae1fa485883issyg
biders saved: d600dc03b11e7d782e1e4dae091b084a1h5ch
biders saved: fe37b8b29d26021ea9462bcc7c5bf5c2pbya7
biders saved: 990db0dcdb665fb8fda5845b3e5d63082mbqq
biders saved: 1a9c554b044c77239b63636008d353d4vy9kh
biders saved: ffbc0fdfbf19a8a9116b68714138f2902cc13
biders saved: c4856fd5abe8f6d6dea36ca2fec444faauos8
biders saved: d174177146370aacbd5ae107b6be78828tx3e
biders saved: 6b09f3376045ebad1504abe3b05c9c943aezq
biders saved: d47110cdd3eeb1ec25659f4285730e0a28ofd
biders saved: c709b6c05ebd88124fe639a70103c8aerho8s
biders saved: 0dea01e325acccb9d2e7cdbf0150619c6aycy
biders saved: 708ba5d36271639a9003ace4dc2cc540mugek
biders saved: 45972f59df79953d9bcf04a76dbe0f51z23qd
biders saved

biders saved: 6c4f92086930144fc55a79dc0a4a6f339zt0p
biders saved: 47efdea29f084d75516f475ee2e21695ira9g
biders saved: 48fd18427f5e4b8d9cb568aa5cd6a3f9sizo0
biders saved: 0cea6e30d0b3e33b2a77d68b8c8e1ab3vzomo
biders saved: a51eb0674d176d1601c239e290d8db9aygqcy
biders saved: bfba91e586b1c37429d1e40ffb5febfagbbxn
biders saved: 5e9beac339e73e3feb340f9372b4aa07l69vw
biders saved: 0565d7d345ac29442b26cd9ffe2b3b841wwnb
biders saved: b569644e65d437c29a86d6c86f86c720gieux
biders saved: 84c3f1ef08df383bdc98f5465e885b6edepz9
biders saved: 55dc05893025427ce0aa0ffce381a2bb5tlqu
biders saved: e0fa75eb43b7bd390221db66926f7563hgapj
biders saved: c02f06b962414bde7331252da5e0ee7dbhxwl
biders saved: cb08091907558694c82474f05fa81567pdtie
biders saved: b57f6875839b5a6ce409198cab6d3d08zoydm
biders saved: 59f61cb410ac321a795c4d5a388e787aqsytg
biders saved: 1a1d4e9eba20c58b73fd68c5ab9907081xw9l
biders saved: 31c469698f66a5d79f5984ab04429837z5ype
biders saved: a64afbb3c053e8cccc58ce09a2e71a5205u6z
biders saved

biders saved: c671c7faec19e9409eed917213831d0015juj
biders saved: b766fbc1d5d0525e18b094162b543b15ub8qf
biders saved: 24d0694fbff381cdd9f40301080301d8yngb7
biders saved: 51f47f594971d59e2a52398527ce8d81q5rdo
biders saved: 6ceea2e62f9c4f03f1a4efa7f8d2b2416hers
biders saved: ecf6ad2bbfa8aafbd7ad51d5bfcc02ddyznwh
biders saved: 78274148071dfc23b583be546d1e80c7lu8rq
biders saved: c1e8091db6fd2c7977b15830e0efc4012zjt0
biders saved: b4259cbae2d1542752b49b96c544be2bxp8u2
biders saved: 02518a364d87661eed1453e251933862yvmjj
biders saved: 0764308c5a1d35bf76c32d943eff07a50tp9e
biders saved: ab7cc752f0ed98d7f8c85e02dbbc4772z9xpx
biders saved: 7e4a99f7ae5fe3a098c9c3f21b9a5cbfa4w4t
biders saved: 3de7d3030fecf543874081f7ab0a248evqper
biders saved: b8fc67ed5825409a40adca46ee7f053auho77
biders saved: 6951db6168b80060a103ab65d0e074484c1vc
biders saved: 9a51d309c8824bae38bd75eddce1a99fe208r
biders saved: b02df28d2798f0b6c2b1112a9d407ae7wbj8r
biders saved: a9c0cd61e6b52a5b4bfb88e5ccb40af36qzu2
biders saved

biders saved: 64b2a3d72fd485989bff7ee11bf66f7b51j0z
biders saved: d8b3e32a6f8436acfe9984227c9d1e57qemx2
biders saved: a351fbee43a761cef22150a06645b52c1k104
biders saved: 336f98d51f267347d9a751d1d7b97355nw7xc
biders saved: 58ea3d35765cb8ceeb6b4be54d754ea0bpguj
biders saved: fe84da5e6912534549d63415ed725618uiz26
biders saved: 554a2b2d9728c432126398b45fe804e6ega9f
biders saved: f8e3d1df88d8cf283a58d0a2329b79a9ilvmp
biders saved: a97593cabdfc92d0ea142d1c64f0cb70aa6cs
biders saved: 33dc4889cc2bd50dab02344d6fe19a1aj59pt
biders saved: 51d6b173586fd34c938477d87ad6b6b0092ke
biders saved: 51b2d173034d54f115b647ac7c1640a5l1q91
biders saved: 3cccf376d4318df23f4bd9321cd83cf0ys9lz
biders saved: 6264a918aa934905a79a167af1d0787e7om3b
biders saved: 8a2b950ec6f3c73253cd80133d184cdawzip0
biders saved: 2974f614bc988411be383b2d6d52003fdb1ps
biders saved: 7fe504252783ec1fd2b203d3de7040e6lflsb
biders saved: f8e215d416736fdb1de26d34ad315c368o24w
biders saved: 59998f2c7299efed20dbb68df23a53c1cdoje
biders saved

...

biders saved: 91cb97491f3deb60457041ad665a642720z5z
biders saved: 8506033bad90a03d59f849054e5a0b91a5gel
biders saved: 601fffaf4ead097c9dc9c40a2eecb505lcs19
biders saved: 620492932ce60827664b417eed6c83d6b1ky0
biders saved: 38c2be4dfa8f0512b3164fbbf6ffb1f6weo0v
biders saved: 82be315206c48aacefccbb7e7da192e2q18am
biders saved: 038d5c12a210df3818eb8dc34ebedac2ldr62
biders saved: 5a12f5a72db38165a610f73387c29ad9v0atb
biders saved: 928996e2dbbae8694d3650587cb49b1ahqbaz
biders saved: fb294f3671fc19eacf5f13d3a6c5a795pzlxm
biders saved: 8c07083331a97da1b8a99cea75775401m0a0r
biders saved: 75351c0b43c8732ed763e9ebc7a3241do08lc
biders saved: 4f8188d297e19e277a2dbafbecdeb53crpf4h
biders saved: 42d68de5fac9c929d58d277a6f1287e0qbql4
biders saved: ade3651ac046baabe9a0d2d4fc587320tlsd1
biders saved: ad69c8ef95b0ca2ad4ab8f9135001030xswpf
biders saved: 1f52c4c6fffd6b5d0f89126c2fa0b45a1l33z
biders saved: f4832b1f7ea28da0093ba42b06e38aa2p386e
biders saved: a8217f09ac37688d06a46dde0a443564dn51e
biders saved

biders saved: 9793f7cb1de19b71abc91ab7a708e4d061o0o
biders saved: 0918faabadfe856a25ebf1ff96b2cfaf7ez6a
biders saved: a20b69d2d198f83727b80db4d74c4018q1kll
biders saved: 1110df63854d2978e639dfb6a0f3ac082gred
biders saved: f97d3883cb3340591418d95f95e81263f6b8v
biders saved: c94233a835d5be66a22073ff18a74fee9qszc
biders saved: f1f5606c00a9a7134a424e0df5402b601m3cm
biders saved: 96f428c4d51b38d861357992871f4a0e8e23b
biders saved: 227aa294b56aea25330e5dc3a6fddb82oq7dn
biders saved: 39d8059f91d70ee5a68731eacec9e104n956u
biders saved: 386b0a2e114d3b617583eff0ef980e67x64mp
biders saved: 7ac5fefc37ae047a92a6b73732c0a96cufh25
biders saved: 2a536c1561bcb52ee0ea03813fa002dczyjhp
biders saved: 493588b5f3249d6ca3c552875b09e7e1ye0v0
biders saved: f71560dbf5f37a2aac1f77fe1771ce6eqxxw5
biders saved: 9ee5ff6c842f7f15b585387a2cfab030fisgm
biders saved: fd40f42b7c5897c817e4063849b18ba11s87k
biders saved: 300246f85d3ec66ed254b1a4be4057492c61l
biders saved: 9ce37c1aab0b14720144bd27cdcad361zg7oo
biders saved

biders saved: 4ccde027a20264ab2d04392b60e31b4fu3ray
biders saved: a356163fe7434d47b1c6480c87366394oaibe
biders saved: da8abe1879eb50004c6276c94b792e11lq4xn
biders saved: a852b00d87721f8d3ca2e1041c088aa507iso
biders saved: 58cbeb46c36b90a723cded01495d055bmwudo
biders saved: 10a1f1cb1fdd93a139efba25087dbeeaulogd
biders saved: 980a1950c0b5df2b9ed03cb05e657848p76xr
biders saved: 992737f67ac0150c8e1448bdc06e3d2f6f57s
biders saved: d88f8b2015441d1ca588f5bc850629040hdtv
biders saved: 54f0943d5e936acdef05bf151ffbb1eczg2ck
biders saved: 8950b00903c06ced5253da31b99d80fe9suie
biders saved: 5ba001c8d950b2b21c465f1fc5e40630qy65o
biders saved: c28049769ec09eb046314cffba0975cfqvkyv
biders saved: b7e9ad9b90c4b148a9ddd7815497f8395rib9
biders saved: cc1b98265a249b0373d3014fc504b100cl1ja
biders saved: 628571f862245c6a83623648b5543d087nfqx
biders saved: 1eb1f5561dd35f544202c3b509d578422058h
biders saved: 740cce529e3b68bb776f3ae276b959bary8ax
biders saved: 0933e1844ef295b026b5e6ff6f4c0960jhzqr
biders saved

biders saved: fdd71714ea1426c36bbc80a157f50c8daex1p
biders saved: e8a1dc9fb527faf420bd7e5196d875cfed9id
biders saved: e154f640ac6b7e56dc4eaa610a283d8e3l2km
biders saved: 601bbfc2d687f56d1b18c99ffde76c7a94ek8
biders saved: 1170c87a5f74dccb65eb064aaffe3a29ioc7n
biders saved: bb75daa2552fa7e03972a7ebd49c8270dxs87
biders saved: 242e4323f41d0a9011b4140ba74f478dnfsqs
biders saved: 1960e29a911e5a2dbc7eba2b373c1e17tjpd0
biders saved: 288452bc92ad126755ead4406718da67e1d38
biders saved: 774e41491bfb8338b75016cf9fba5d9e6oho5
biders saved: b972f198aad37412a797c0e9b5ef232ac2fhl
biders saved: 222a84e78164c6fbe3cc7e7fc25c16b2jqncf
biders saved: 06052ab145f8a3ddc6ad8433ef11da8dputmu
biders saved: bd612f236a50b7028858ec6a9518dcf82wdtq
biders saved: c979898ec4012f0933c38a6bec19c92cvmlq6
biders saved: 05f5792216f9a5bfe91ecd0dbccc3cdceuwmf
biders saved: 016e3c511caf01a845c9c976bbf355a6m1nns
biders saved: 6a7fc08059bc5d4ea71f7a00db4d6756xuba2
biders saved: 5ab20c9699e9ea7c9b395967027b3245f4uq1
biders saved

biders saved: 6887f0abc4eb4c79eb0e23c48ceea186vjfih
biders saved: 5455e2e995dbd8e57ced16096e5a2769lmthu
biders saved: 605520624bbf2020903a32aecd36e535jgj0h
biders saved: 2d0703227815eb74617f639b13460351i3s9l
biders saved: bec86221a3ab9f5b8f502a4bed42f2cfddf3p
biders saved: 1b2e5adebf42f673c117d008cc713ff1x6cs4
biders saved: 0751502caf4afd959c99955c80edacc85u11v
biders saved: 61665d0c0ac353aa4f429f636879af8bmuf9n
biders saved: 3a2e6d8da93da7b6f5b9c6ba770f8687tfkyk
biders saved: 1af3a143e126acfc43031dd8e4612b2egr927
biders saved: 83b6cbf8c26ea604b48f508dd7728c9414pfn
biders saved: bd890937b5601111119969b45bde5056eycor
biders saved: 107f0f8f2bf4fc62ba8c01003c516c03npt0l
biders saved: 9f9313a1d8a88b3a3c8a2d893a92820dn38qc
biders saved: 5563598101c7e6533b63f26827e57aa96qjww
biders saved: 15e7068e841df3f00857b9b1db46e79200trv
biders saved: 5a8019da4075c77405e43efd072670eegbme0
biders saved: 9390ac0a97e75b3b56fe63695d27a82euhv69
biders saved: 13eaa629e1a8fd178403a66fcf0758bbirawk
biders saved